# 1. SETTINGS

In [27]:
# libraries
import pandas as pd
import numpy as np
import scipy.stats
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt

In [28]:
# pandas options
pd.set_option("display.max_columns", None)

In [29]:
# ignore warnings
import warnings
warnings.filterwarnings("ignore")

In [30]:
# garbage collection
import gc
gc.enable()

# 2. FUNCTIONS

In [31]:
##### FUNCTION FOR COUNTING MISSINGS
def count_missings(data):
    total = data.isnull().sum().sort_values(ascending = False)
    percent = (data.isnull().sum() / data.isnull().count() * 100).sort_values(ascending = False)
    table = pd.concat([total, percent], axis = 1, keys = ["Total", "Percent"])
    table = table[table["Total"] > 0]
    return table

In [32]:
##### FUNCTION FOR CREATING LOGARITHMS
def convert_days(data, features, t = 12, rounding = True, replace = False):
    for var in features:
        if replace == True:
            if rounding == True:
                data[var] = round(-data[var]/t)
            else:
                data[var] = -data[var]/t
            data[var][data[var] < 0] = None
        else:
            if rounding == True:
                data["CONVERTED_" + str(var)] = round(-data[var]/t)
            else:
                data["CONVERTED_" + str(var)] = -data[var]/t
            data["CONVERTED_" + str(var)][data["CONVERTED_" + str(var)] < 0] = None
    return data

In [33]:
##### FUNCTION FOR CREATING LOGARITHMS
def create_logs(data, features, replace = False):
    for var in features:
        if replace == True:
            data[var] = np.log(data[var].abs() + 1)
        else:
            data["LOG_" + str(var)] = np.log(data[var].abs() + 1)      
    return data

In [34]:
##### FUNCTION FOR CREATING FLAGS FOR MISSINGS
def create_null_flags(data, features = None):
    if features == None:
        features = data.columns
    for var in features:
        num_null = data[var].isnull() + 0
        if num_null.sum() > 0:
            data["ISNULL_" + str(var)] = num_null
    return data

In [35]:
##### FUNCTION FOR TREATING FACTORS
def treat_factors(data, method = "label"):
    
    # label encoding
    if method == "label":
        factors = [f for f in data.columns if data[f].dtype == "object"]
        for var in factors:
            data[var], _ = pd.factorize(data[var])
        
    # dummy encoding
    if method == "dummy":
        data = pd.get_dummies(data, drop_first = True)
    
    # dataset
    return data

In [36]:
##### FUNCTION FOR COMPUTING ACCEPT/REJECT RATIOS
def compute_accept_reject_ratio(data, lags = [1, 3, 5]):
    
    # preparations
    dec_prev = data[["SK_ID_CURR", "SK_ID_PREV", "DAYS_DECISION", "NAME_CONTRACT_STATUS"]]
    dec_prev["DAYS_DECISION"] = -dec_prev["DAYS_DECISION"]
    dec_prev = dec_prev.sort_values(by = ["SK_ID_CURR", "DAYS_DECISION"])
    dec_prev = pd.get_dummies(dec_prev)
     
    # compuatation
    for t in lags:
        
        # acceptance ratios
        tmp = dec_prev[["SK_ID_CURR", "NAME_CONTRACT_STATUS_Approved"]].groupby(["SK_ID_CURR"]).head(1)
        tmp = tmp.groupby(["SK_ID_CURR"], as_index = False).mean()
        tmp.columns = ["SK_ID_CURR", "APPROVE_RATIO_" + str(t)]
        data = data.merge(tmp, how = "left", on = "SK_ID_CURR")
        
        # rejection ratios
        tmp = dec_prev[["SK_ID_CURR", "NAME_CONTRACT_STATUS_Refused"]].groupby(["SK_ID_CURR"]).head(1)
        tmp = tmp.groupby(["SK_ID_CURR"], as_index = False).mean()
        tmp.columns = ["SK_ID_CURR", "REJECT_RATIO_" + str(t)]
        data = data.merge(tmp, how = "left", on = "SK_ID_CURR")
        
    # dataset
    return data

In [37]:
##### FUNCTION FOR AGGREGATING DATA
def aggregate_data(data, id_var, label = None):
    
    
    ### SEPARATE FEATURES
  
    # display info
    print("- Preparing the dataset...")

    # find factors
    data_factors = [f for f in data.columns if data[f].dtype == "object"]
    
    # partition subsets
    num_data = data[list(set(data.columns) - set(data_factors))]
    fac_data = data[[id_var] + data_factors]
    
    # display info
    num_facs = fac_data.shape[1] - 1
    num_nums = num_data.shape[1] - 1
    print("- Extracted %.0f factors and %.0f numerics..." % (num_facs, num_nums))


    ##### AGGREGATION
 
    # aggregate numerics
    if (num_nums > 0):
        print("- Aggregating numeric features...")
        num_data = num_data.groupby(id_var).agg(["mean", "std", "min", "max"])
        num_data.columns = ["_".join(col).strip() for col in num_data.columns.values]
        num_data = num_data.sort_index()

    # aggregate factors
    if (num_facs > 0):
        print("- Aggregating factor features...")
        fac_data = fac_data.groupby(id_var).agg([("mode",   lambda x: scipy.stats.mode(x)[0][0]),
                                                 ("unique", lambda x: x.nunique())])
        fac_data.columns = ["_".join(col).strip() for col in fac_data.columns.values]
        fac_data = fac_data.sort_index()


    ##### MERGER

    # merge numerics and factors
    if ((num_facs > 0) & (num_nums > 0)):
        agg_data = pd.concat([num_data, fac_data], axis = 1)
    
    # use factors only
    if ((num_facs > 0) & (num_nums == 0)):
        agg_data = fac_data
        
    # use numerics only
    if ((num_facs == 0) & (num_nums > 0)):
        agg_data = num_data
        

    ##### LAST STEPS

    # update labels
    if label != None:
        agg_data.columns = [label + "_" + str(col) for col in agg_data.columns]
    
    # impute zeros for SD
    #stdevs = agg_data.filter(like = "_std").columns
    #for var in stdevs:
    #    agg_data[var].fillna(0, inplace = True)

    # display info
    print("- Final dimensions:", agg_data.shape)
    
    # return dataset
    return agg_data

# 3. DATA IMPORT

In [38]:
# import data
train = pd.read_csv("../data/raw/application_train.csv")
test  = pd.read_csv("../data/raw/application_test.csv")
buro  = pd.read_csv("../data/raw/bureau.csv")
bbal  = pd.read_csv("../data/raw/bureau_balance.csv")
prev  = pd.read_csv("../data/raw/previous_application.csv")
card  = pd.read_csv("../data/raw/credit_card_balance.csv")
poca  = pd.read_csv("../data/raw/POS_CASH_balance.csv")
inst  = pd.read_csv("../data/raw/installments_payments.csv")

In [39]:
# check dimensions
print("Application:", train.shape, test.shape)
print("Buro:", buro.shape)
print("Bbal:", bbal.shape)
print("Prev:", prev.shape)
print("Card:", card.shape)
print("Poca:", poca.shape)
print("Inst:", inst.shape)

Application: (307511, 122) (48744, 121)
Buro: (1716428, 17)
Bbal: (27299925, 3)
Prev: (1670214, 37)
Card: (3840312, 23)
Poca: (10001358, 8)
Inst: (13605401, 8)


In [40]:
# extract target
y = train[["SK_ID_CURR", "TARGET"]]
del train["TARGET"]

# 4. PREPROCESSING

## 4.1. APPLICATION DATA

In [41]:
# concatenate application data
appl = pd.concat([train, test])
del train, test

In [42]:
### FEATURE ENGINEERING

# income ratios
appl["CREDIT_BY_INCOME"]      = appl["AMT_CREDIT"]       / appl["AMT_INCOME_TOTAL"]
appl["ANNUITY_BY_INCOME"]     = appl["AMT_ANNUITY"]      / appl["AMT_INCOME_TOTAL"]
appl["GOODS_PRICE_BY_INCOME"] = appl["AMT_GOODS_PRICE"]  / appl["AMT_INCOME_TOTAL"]
appl["INCOME_PER_PERSON"]     = appl["AMT_INCOME_TOTAL"] / appl["CNT_FAM_MEMBERS"]

# career ratio
appl["PERCENT_WORKED"] = appl["DAYS_EMPLOYED"] / appl["DAYS_BIRTH"]
appl["PERCENT_WORKED"][appl["PERCENT_WORKED"] < 0] = None

# number of adults
appl["CNT_ADULTS"] = appl["CNT_FAM_MEMBERS"] - appl["CNT_CHILDREN"]

# external sources
appl["EXT_SOURCE_MEAN"] = appl[["EXT_SOURCE_1", "EXT_SOURCE_1", "EXT_SOURCE_3"]].mean(axis = 1)
appl["EXT_SOURCE_SD"]   = appl[["EXT_SOURCE_1", "EXT_SOURCE_1", "EXT_SOURCE_3"]].std(axis = 1)
appl["NUM_EXT_SOURCES"] = 3 - (appl["EXT_SOURCE_1"].isnull().astype(int) +
                               appl["EXT_SOURCE_2"].isnull().astype(int) +
                               appl["EXT_SOURCE_3"].isnull().astype(int))

# number of documents
doc_vars = ["FLAG_DOCUMENT_2",  "FLAG_DOCUMENT_3",  "FLAG_DOCUMENT_4",  "FLAG_DOCUMENT_5",  "FLAG_DOCUMENT_6",
            "FLAG_DOCUMENT_7",  "FLAG_DOCUMENT_8",  "FLAG_DOCUMENT_9",  "FLAG_DOCUMENT_10", "FLAG_DOCUMENT_11",
            "FLAG_DOCUMENT_12", "FLAG_DOCUMENT_13", "FLAG_DOCUMENT_14", "FLAG_DOCUMENT_15", "FLAG_DOCUMENT_16",
            "FLAG_DOCUMENT_17", "FLAG_DOCUMENT_18", "FLAG_DOCUMENT_19", "FLAG_DOCUMENT_20", "FLAG_DOCUMENT_21"]
appl["NUM_DOCUMENTS"] = appl[doc_vars].sum(axis = 1)

# application date
appl["DAY_APPR_PROCESS_START"] = "Working day"
appl["DAY_APPR_PROCESS_START"][(appl["WEEKDAY_APPR_PROCESS_START"] == "SATURDAY") |
                               (appl["WEEKDAY_APPR_PROCESS_START"] == "SUNDAY")] = "Weekend"

# logarithms
log_vars = ["AMT_CREDIT", "AMT_INCOME_TOTAL", "AMT_GOODS_PRICE", "AMT_ANNUITY"]
appl = create_logs(appl, log_vars, replace = True)

# convert days
day_vars = ["DAYS_BIRTH", "DAYS_REGISTRATION", "DAYS_ID_PUBLISH", "DAYS_EMPLOYED", "DAYS_LAST_PHONE_CHANGE"]
appl = convert_days(appl, day_vars, t = 30, rounding = True, replace = True)

# age ratios
appl["OWN_CAR_AGE_RATIO"] = appl["OWN_CAR_AGE"] / appl["DAYS_BIRTH"]
appl["DAYS_ID_PUBLISHED_RATIO"] = appl["DAYS_ID_PUBLISH"] / appl["DAYS_BIRTH"]
appl["DAYS_REGISTRATION_RATIO"] = appl["DAYS_REGISTRATION"] / appl["DAYS_BIRTH"]
appl["DAYS_LAST_PHONE_CHANGE_RATIO"] = appl["DAYS_LAST_PHONE_CHANGE"] / appl["DAYS_BIRTH"]


##### FEATURE REMOVAL
drops = ['APARTMENTS_MEDI', 'BASEMENTAREA_MEDI', 'COMMONAREA_MEDI', 'ELEVATORS_MEDI', 'ENTRANCES_MEDI', 
         'FLOORSMAX_MEDI', 'FLOORSMIN_MEDI', 'LANDAREA_MEDI', 'LIVINGAPARTMENTS_MEDI', 'LIVINGAREA_MEDI',
         'NONLIVINGAPARTMENTS_MEDI', 'NONLIVINGAREA_MEDI','YEARS_BEGINEXPLUATATION_MEDI', 'YEARS_BUILD_MEDI',
         'APARTMENTS_MODE', 'BASEMENTAREA_MODE', 'COMMONAREA_MODE','ELEVATORS_MODE', 'ENTRANCES_MODE', 
         'FLOORSMAX_MODE', 'FLOORSMIN_MODE', 'LANDAREA_MODE', 'LIVINGAPARTMENTS_MODE', 'LIVINGAREA_MODE', 
         'NONLIVINGAPARTMENTS_MODE', 'NONLIVINGAREA_MODE', 'TOTALAREA_MODE',  'YEARS_BEGINEXPLUATATION_MODE']
appl = appl.drop(columns = drops)

In [43]:
# dummy encodnig for factors
appl = pd.get_dummies(appl, drop_first = True)

In [44]:
# rename features
appl.columns = ["SK_ID_CURR"] + ["app_" + str(col) for col in appl.columns if col not in "SK_ID_CURR"]

In [45]:
# check data
appl.head()

,SK_ID_CURR,app_CNT_CHILDREN,app_AMT_INCOME_TOTAL,app_AMT_CREDIT,app_AMT_ANNUITY,app_AMT_GOODS_PRICE,app_REGION_POPULATION_RELATIVE,app_DAYS_BIRTH,app_DAYS_EMPLOYED,app_DAYS_REGISTRATION,app_DAYS_ID_PUBLISH,app_OWN_CAR_AGE,app_FLAG_MOBIL,app_FLAG_EMP_PHONE,app_FLAG_WORK_PHONE,app_FLAG_CONT_MOBILE,app_FLAG_PHONE,app_FLAG_EMAIL,app_CNT_FAM_MEMBERS,app_REGION_RATING_CLIENT,app_REGION_RATING_CLIENT_W_CITY,app_HOUR_APPR_PROCESS_START,app_REG_REGION_NOT_LIVE_REGION,app_REG_REGION_NOT_WORK_REGION,app_LIVE_REGION_NOT_WORK_REGION,app_REG_CITY_NOT_LIVE_CITY,app_REG_CITY_NOT_WORK_CITY,app_LIVE_CITY_NOT_WORK_CITY,app_EXT_SOURCE_1,app_EXT_SOURCE_2,app_EXT_SOURCE_3,app_APARTMENTS_AVG,app_BASEMENTAREA_AVG,app_YEARS_BEGINEXPLUATATION_AVG,app_YEARS_BUILD_AVG,app_COMMONAREA_AVG,app_ELEVATORS_AVG,app_ENTRANCES_AVG,app_FLOORSMAX_AVG,app_FLOORSMIN_AVG,app_LANDAREA_AVG,app_LIVINGAPARTMENTS_AVG,app_LIVINGAREA_AVG,app_NONLIVINGAPARTMENTS_AVG,app_NONLIVINGAREA_AVG,app_YEARS_BUILD_MODE,app_OBS_30_CNT_SOCIAL_CIRCLE,app_DEF_30_CNT_SOCIAL_CIRCLE,app_OBS_60_CNT_SOCIAL_CIRCLE,app_DEF_60_CNT_SOCIAL_CIRCLE,app_DAYS_LAST_PHONE_CHANGE,app_FLAG_DOCUMENT_2,app_FLAG_DOCUMENT_3,app_FLAG_DOCUMENT_4,app_FLAG_DOCUMENT_5,app_FLAG_DOCUMENT_6,app_FLAG_DOCUMENT_7,app_FLAG_DOCUMENT_8,app_FLAG_DOCUMENT_9,app_FLAG_DOCUMENT_10,app_FLAG_DOCUMENT_11,app_FLAG_DOCUMENT_12,app_FLAG_DOCUMENT_13,app_FLAG_DOCUMENT_14,app_FLAG_DOCUMENT_15,app_FLAG_DOCUMENT_16,app_FLAG_DOCUMENT_17,app_FLAG_DOCUMENT_18,app_FLAG_DOCUMENT_19,app_FLAG_DOCUMENT_20,app_FLAG_DOCUMENT_21,app_AMT_REQ_CREDIT_BUREAU_HOUR,app_AMT_REQ_CREDIT_BUREAU_DAY,app_AMT_REQ_CREDIT_BUREAU_WEEK,app_AMT_REQ_CREDIT_BUREAU_MON,app_AMT_REQ_CREDIT_BUREAU_QRT,app_AMT_REQ_CREDIT_BUREAU_YEAR,app_CREDIT_BY_INCOME,app_ANNUITY_BY_INCOME,app_GOODS_PRICE_BY_INCOME,app_INCOME_PER_PERSON,app_PERCENT_WORKED,app_CNT_ADULTS,app_EXT_SOURCE_MEAN,app_EXT_SOURCE_SD,app_NUM_EXT_SOURCES,app_NUM_DOCUMENTS,app_OWN_CAR_AGE_RATIO,app_DAYS_ID_PUBLISHED_RATIO,app_DAYS_REGISTRATION_RATIO,app_DAYS_LAST_PHONE_CHANGE_RATIO,app_NAME_CONTRACT_TYPE_Revolving loans,app_CODE_GENDER_M,app_CODE_GENDER_XNA,app_FLAG_OWN_CAR_Y,app_FLAG_OWN_REALTY_Y,app_NAME_TYPE_SUITE_Family,app_NAME_TYPE_SUITE_Group of people,app_NAME_TYPE_SUITE_Other_A,app_NAME_TYPE_SUITE_Other_B,"app_NAME_TYPE_SUITE_Spouse, partner",app_NAME_TYPE_SUITE_Unaccompanied,app_NAME_INCOME_TYPE_Commercial associate,app_NAME_INCOME_TYPE_Maternity leave,app_NAME_INCOME_TYPE_Pensioner,app_NAME_INCOME_TYPE_State servant,app_NAME_INCOME_TYPE_Student,app_NAME_INCOME_TYPE_Unemployed,app_NAME_INCOME_TYPE_Working,app_NAME_EDUCATION_TYPE_Higher education,app_NAME_EDUCATION_TYPE_Incomplete higher,app_NAME_EDUCATION_TYPE_Lower secondary,app_NAME_EDUCATION_TYPE_Secondary / secondary special,app_NAME_FAMILY_STATUS_Married,app_NAME_FAMILY_STATUS_Separated,app_NAME_FAMILY_STATUS_Single / not married,app_NAME_FAMILY_STATUS_Unknown,app_NAME_FAMILY_STATUS_Widow,app_NAME_HOUSING_TYPE_House / apartment,app_NAME_HOUSING_TYPE_Municipal apartment,app_NAME_HOUSING_TYPE_Office apartment,app_NAME_HOUSING_TYPE_Rented apartment,app_NAME_HOUSING_TYPE_With parents,app_OCCUPATION_TYPE_Cleaning staff,app_OCCUPATION_TYPE_Cooking staff,app_OCCUPATION_TYPE_Core staff,app_OCCUPATION_TYPE_Drivers,app_OCCUPATION_TYPE_HR staff,app_OCCUPATION_TYPE_High skill tech staff,app_OCCUPATION_TYPE_IT staff,app_OCCUPATION_TYPE_Laborers,app_OCCUPATION_TYPE_Low-skill Laborers,app_OCCUPATION_TYPE_Managers,app_OCCUPATION_TYPE_Medicine staff,app_OCCUPATION_TYPE_Private service staff,app_OCCUPATION_TYPE_Realty agents,app_OCCUPATION_TYPE_Sales staff,app_OCCUPATION_TYPE_Secretaries,app_OCCUPATION_TYPE_Security staff,app_OCCUPATION_TYPE_Waiters/barmen staff,app_WEEKDAY_APPR_PROCESS_START_MONDAY,app_WEEKDAY_APPR_PROCESS_START_SATURDAY,app_WEEKDAY_APPR_PROCESS_START_SUNDAY,app_WEEKDAY_APPR_PROCESS_START_THURSDAY,app_WEEKDAY_APPR_PROCESS_START_TUESDAY,app_WEEKDAY_APPR_PROCESS_START_WEDNESDAY,app_ORGANIZATION_TYPE_Agriculture,app_ORGANIZATION_TYPE_Bank,app_ORGANIZATION_T

In [46]:
# count missings
nas = count_missings(appl)
nas.head()

,Total,Percent
app_COMMONAREA_AVG,248360,69.714109
app_NONLIVINGAPARTMENTS_AVG,246861,69.293343
app_LIVINGAPARTMENTS_AVG,242979,68.203674
app_FLOORSMIN_AVG,241108,67.678489
app_YEARS_BUILD_AVG,236306,66.330578


## 4.2. CREDIT BUREAU DATA

### 4.2.1. BBAL DATA

In [47]:
# check bbal data
bbal.head()

,SK_ID_BUREAU,MONTHS_BALANCE,STATUS
0,5715448,0,C
1,5715448,-1,C
2,5715448,-2,C
3,5715448,-3,C
4,5715448,-4,C


In [48]:
### FEATURE ENGINEERING

# loan default score
bbal["NUM_STATUS"] = 0
bbal["NUM_STATUS"][bbal["STATUS"] == "X"] = None
bbal["NUM_STATUS"][bbal["STATUS"] == "1"] = 1
bbal["NUM_STATUS"][bbal["STATUS"] == "2"] = 2
bbal["NUM_STATUS"][bbal["STATUS"] == "3"] = 3
bbal["NUM_STATUS"][bbal["STATUS"] == "4"] = 4
bbal["NUM_STATUS"][bbal["STATUS"] == "5"] = 5
bbal["LOAN_SCORE"] = bbal["NUM_STATUS"] / (abs(bbal["MONTHS_BALANCE"]) + 1)
loan_score = bbal.groupby("SK_ID_BUREAU", as_index = False).LOAN_SCORE.sum()
del bbal["NUM_STATUS"]
del bbal["LOAN_SCORE"]

# dummy encoding for STATUS
bbal = pd.get_dummies(bbal, columns = ["STATUS"], prefix = "STATUS")

In [49]:
# count missings
nas = count_missings(bbal)
nas.head()

,Total,Percent


In [50]:
### AGGREGATIONS

# total month count
cnt_mon = bbal[["SK_ID_BUREAU", "MONTHS_BALANCE"]].groupby("SK_ID_BUREAU").count()
del bbal["MONTHS_BALANCE"]

# aggregate data
agg_bbal = bbal.groupby("SK_ID_BUREAU").mean()

# add total month count
agg_bbal["MONTH_COUNT"] = cnt_mon

# add loan score
agg_bbal = agg_bbal.merge(loan_score, how = "left", on = "SK_ID_BUREAU")

In [51]:
# count missings
nas = count_missings(agg_bbal)
nas.head()

,Total,Percent


In [52]:
# check data
agg_bbal.head()

,SK_ID_BUREAU,STATUS_0,STATUS_1,STATUS_2,STATUS_3,STATUS_4,STATUS_5,STATUS_C,STATUS_X,MONTH_COUNT,LOAN_SCORE
0,5001709,0.000000,0.0,0.0,0.0,0.0,0.0,0.886598,0.113402,97,0.0
1,5001710,0.060241,0.0,0.0,0.0,0.0,0.0,0.578313,0.361446,83,0.0
2,5001711,0.750000,0.0,0.0,0.0,0.0,0.0,0.000000,0.250000,4,0.0
3,5001712,0.526316,0.0,0.0,0.0,0.0,0.0,0.473684,0.000000,19,0.0
4,5001713,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,1.000000,22,0.0


In [53]:
# clear memory
del bbal

### 4.2.2. BURO DATA

In [54]:
# check buro data
buro.head()

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.0,171342.0,NaN,0.0,Credit card,-20,NaN
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.5,NaN,NaN,0.0,Consumer credit,-16,NaN
3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.0,NaN,NaN,0.0,Credit card,-16,NaN
4,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.0,NaN,NaN,0.0,Consumer credit,-21,NaN


In [55]:
### MERGE
buro = buro.merge(right = bbal.reset_index(), how = "left", on = "SK_ID_BUREAU")

NameError: name 'bbal' is not defined

In [ ]:
##### FEATURE ENGINEERING

# number of buro loans 
cnt_buro = buro[["SK_ID_CURR", "SK_ID_BUREAU"]].groupby(["SK_ID_CURR"], as_index = False).count()
cnt_buro.columns = ["SK_ID_CURR", "CNT_BURO_LOANS"]
buro = buro.merge(cnt_buro, how = "left", on = "SK_ID_CURR")

# amount ratios
buro["AMT_SUM_OVERDUE_RATIO_1"] = buro["AMT_CREDIT_SUM_OVERDUE"] / buro["AMT_ANNUITY"]
buro["AMT_SUM_OVERDUE_RATIO_2"] = buro["AMT_CREDIT_SUM_OVERDUE"] / buro["AMT_CREDIT_SUM"]
buro["AMT_MAX_OVERDUE_RATIO_1"] = buro["AMT_CREDIT_MAX_OVERDUE"] / buro["AMT_ANNUITY"]
buro["AMT_MAX_OVERDUE_RATIO_2"] = buro["AMT_CREDIT_MAX_OVERDUE"] / buro["AMT_CREDIT_SUM"]
buro["AMT_SUM_DEBT_RATIO_1"]    = buro["AMT_CREDIT_SUM_DEBT"] / buro["AMT_CREDIT_SUM"]
buro["AMT_SUM_DEBT_RATIO_2"]    = buro["AMT_CREDIT_SUM_DEBT"] / buro["AMT_CREDIT_SUM_LIMIT"]

# logarithms
log_vars = ["AMT_CREDIT_SUM", "AMT_CREDIT_SUM_DEBT", "AMT_CREDIT_SUM_LIMIT", "AMT_CREDIT_SUM_OVERDUE", "AMT_ANNUITY"]
buro = create_logs(buro, log_vars, replace = True)

# convert days
day_vars = ["DAYS_CREDIT", "CREDIT_DAY_OVERDUE", "DAYS_CREDIT_ENDDATE", "DAYS_ENDDATE_FACT", "DAYS_CREDIT_UPDATE"]
buro = convert_days(buro, day_vars, t = 1, rounding = False, replace = True)

# recency-weighted loan score
buro["WEIGHTED_LOAN_SCORE"] = buro["LOAN_SCORE"] / (buro["DAYS_CREDIT"] / 12)

# day differences
buro["DAYS_END_DIFF_1"] = buro["DAYS_ENDDATE_FACT"]   - buro["DAYS_CREDIT_ENDDATE"]
buro["DAYS_END_DIFF_2"] = buro["DAYS_CREDIT_UPDATE"]  - buro["DAYS_CREDIT_ENDDATE"]
buro["DAYS_DURATION_1"] = buro["DAYS_CREDIT_ENDDATE"] - buro["DAYS_CREDIT"]
buro["DAYS_DURATION_2"] = buro["DAYS_ENDDATE_FACT"]   - buro["DAYS_CREDIT"]

# number of active buro loans
cnt_buro = buro[["SK_ID_CURR", "CREDIT_ACTIVE"]]
cnt_buro.columns = ["SK_ID_CURR", "CNT_BURO_ACTIVE"]
cnt_buro = cnt_buro[cnt_buro["CNT_BURO_ACTIVE"] == "Active"]
cnt_buro = cnt_buro[["SK_ID_CURR", "CNT_BURO_ACTIVE"]].groupby(["SK_ID_CURR"], as_index = False).count()
buro = buro.merge(cnt_buro, how = "left", on = "SK_ID_CURR")
buro["CNT_BURO_ACTIVE"].fillna(0, inplace = True)

# number of closed buro loans
cnt_buro = buro[["SK_ID_CURR", "CREDIT_ACTIVE"]]
cnt_buro.columns = ["SK_ID_CURR", "CNT_BURO_CLOSED"]
cnt_buro = cnt_buro[cnt_buro["CNT_BURO_CLOSED"] == "Closed"]
cnt_buro = cnt_buro[["SK_ID_CURR", "CNT_BURO_CLOSED"]].groupby(["SK_ID_CURR"], as_index = False).count()
buro = buro.merge(cnt_buro, how = "left", on = "SK_ID_CURR")
buro["CNT_BURO_CLOSED"].fillna(0, inplace = True)

# number of defaulted buro loans
cnt_buro = buro[["SK_ID_CURR", "CREDIT_ACTIVE"]]
cnt_buro.columns = ["SK_ID_CURR", "CNT_BURO_BAD"]
cnt_buro = cnt_buro[cnt_buro["CNT_BURO_BAD"] == "Bad debt"]
cnt_buro = cnt_buro[["SK_ID_CURR", "CNT_BURO_BAD"]].groupby(["SK_ID_CURR"], as_index = False).count()
buro = buro.merge(cnt_buro, how = "left", on = "SK_ID_CURR")
buro["CNT_BURO_BAD"].fillna(0, inplace = True)

In [ ]:
# dummy encodnig for factors
buro = pd.get_dummies(buro, drop_first = True)

In [ ]:
# count missings
nas = count_missings(buro)
nas.head()

## 4.3. PREVIOUS LOAN DATA

### 4.3.1. INST DATA

In [ ]:
# check inst data
inst.head()

In [ ]:
### FEATURE ENGINEERING

# days past due and days before due (no negative values)
inst['DPD'] = inst['DAYS_ENTRY_PAYMENT'] - inst['DAYS_INSTALMENT']
inst['DBD'] = inst['DAYS_INSTALMENT'] - inst['DAYS_ENTRY_PAYMENT']
inst['DPD'] = inst['DPD'].apply(lambda x: x if x > 0 else 0)
inst['DBD'] = inst['DBD'].apply(lambda x: x if x > 0 else 0)

# percentage and difference paid in each installment 
inst['PAYMENT_PERC'] = inst['AMT_PAYMENT'] / inst['AMT_INSTALMENT']
inst['PAYMENT_DIFF'] = inst['AMT_INSTALMENT'] - inst['AMT_PAYMENT']

# logarithms
log_vars = ["AMT_INSTALMENT", "AMT_PAYMENT"]
inst = create_logs(inst, log_vars, replace = True)

In [ ]:
# dummy encodnig for factors
inst = pd.get_dummies(inst, drop_first = True)

In [ ]:
# count missings
nas = count_missings(inst)
nas.head()

In [ ]:
### AGGREGATIONS

# count instalments
cnt_inst = inst[["SK_ID_PREV", "NUM_INSTALMENT_NUMBER"]].groupby("SK_ID_PREV").count()
del inst["NUM_INSTALMENT_NUMBER"]

# delete ID_CURR
inst_id = inst[["SK_ID_CURR", "SK_ID_PREV"]]
del inst["SK_ID_CURR"]

# aggregate data
agg_inst = aggregate_data(inst, id_var = "SK_ID_PREV")

# add instalment count
agg_inst["inst_INST_COUNT"] = cnt_inst

In [ ]:
# count missings
nas = count_missings(agg_inst)
nas.head()

In [ ]:
# check data
agg_inst.head()

In [ ]:
# clear memory
del inst

### 4.3.2. POCA DATA

In [ ]:
# check poca data
poca.head()

In [ ]:
### FEATURE ENGINEERING

# installments percentage
poca["INSTALLMENTS_PERCENT"] = poca["CNT_INSTALMENT_FUTURE"] / poca["CNT_INSTALMENT"]

In [ ]:
# dummy encodnig for factors
poca = pd.get_dummies(poca, drop_first = True)

In [ ]:
# count missings
nas = count_missings(poca)
nas.head()

In [ ]:
### AGGREGATIONS

# count months
cnt_mon = poca[["SK_ID_PREV", "MONTHS_BALANCE"]].groupby("SK_ID_PREV").count()
del poca["MONTHS_BALANCE"]

# delete ID_CURR
poca_id = poca[["SK_ID_CURR", "SK_ID_PREV"]]
del poca["SK_ID_CURR"]

# aggregate data
agg_poca = aggregate_data(poca, id_var = "SK_ID_PREV")

# add month count
agg_poca["poca_MON_COUNT"] = cnt_mon

In [ ]:
# count missings
nas = count_missings(agg_poca)
nas.head()

In [ ]:
# check data
agg_poca.head()

In [ ]:
# clear memory
del poca

### 4.3.3. CARD DATA

In [ ]:
# check card data
card.head()

In [ ]:
### FEATURE ENGINEERING

# logarithms
log_vars = ["AMT_BALANCE", "AMT_CREDIT_LIMIT_ACTUAL", "AMT_DRAWINGS_ATM_CURRENT", "AMT_DRAWINGS_CURRENT",
            "AMT_DRAWINGS_OTHER_CURRENT", "AMT_DRAWINGS_POS_CURRENT", "AMT_INST_MIN_REGULARITY",
            "AMT_PAYMENT_CURRENT", "AMT_PAYMENT_TOTAL_CURRENT", "AMT_RECEIVABLE_PRINCIPAL",
            "AMT_RECIVABLE", "AMT_TOTAL_RECEIVABLE"]
card = create_logs(card, log_vars, replace = True)

In [ ]:
# dummy encodnig for factors
card = pd.get_dummies(card, drop_first = True)

In [ ]:
# count missings
nas = count_missings(card)
nas.head()

In [ ]:
### AGGREGATIONS

# count months
cnt_mon = card[["SK_ID_PREV", "MONTHS_BALANCE"]].groupby("SK_ID_PREV").count()
del card["MONTHS_BALANCE"]

# delete ID_CURR
card_id = card[["SK_ID_CURR", "SK_ID_PREV"]]
del card["SK_ID_CURR"]

# aggregate data
agg_card = aggregate_data(card, id_var = "SK_ID_PREV")

# add month count
agg_card["card_MON_COUNT"] = cnt_mon

In [ ]:
# count missings
nas = count_missings(agg_card)
nas.head()

In [ ]:
# check data
agg_card.head()

In [ ]:
# clear memory
del card

### 4.3.4. PREV DATA

In [ ]:
# check card data
prev.head()

In [ ]:
### FEATURE ENGINEERING

# amount ratios
prev["AMT_GIVEN_RATIO_1"]  = prev["AMT_CREDIT"] / prev["AMT_APPLICATION"]
prev["AMT_GIVEN_RATIO_2"]  = prev["AMT_GOODS_PRICE"] / prev["AMT_APPLICATION"]
prev["DOWN_PAYMENT_RATIO"] = prev["AMT_DOWN_PAYMENT"] / prev["AMT_APPLICATION"]

# logarithms
log_vars = ["AMT_CREDIT", "AMT_ANNUITY", "AMT_APPLICATION", "AMT_DOWN_PAYMENT", "AMT_GOODS_PRICE"]
prev = create_logs(prev, log_vars, replace = True)

# convert days
day_vars = ["DAYS_FIRST_DRAWING", "DAYS_FIRST_DUE", "DAYS_LAST_DUE_1ST_VERSION", 
            "DAYS_LAST_DUE", "DAYS_TERMINATION", "DAYS_DECISION"]
prev = convert_days(prev, day_vars, t = 1, rounding = False, replace = True)

# number of applications 
cnt_prev = prev[["SK_ID_CURR", "SK_ID_PREV"]].groupby(["SK_ID_CURR"], as_index = False).count()
cnt_prev.columns = ["SK_ID_CURR", "CNT_PREV_APPLICATIONS"]
prev = prev.merge(cnt_prev, how = "left", on = "SK_ID_CURR")

# number of contracts
cnt_prev = prev[["SK_ID_CURR", "FLAG_LAST_APPL_PER_CONTRACT"]]
cnt_prev.columns = ["SK_ID_CURR", "CNT_PREV_CONTRACTS"]
cnt_prev = cnt_prev[cnt_prev["CNT_PREV_CONTRACTS"] == "Y"]
cnt_prev = cnt_prev[["SK_ID_CURR", "CNT_PREV_CONTRACTS"]].groupby(["SK_ID_CURR"], as_index = False).count()
prev = prev.merge(cnt_prev, how = "left", on = "SK_ID_CURR")

# number ratio
prev["APPL_PER_CONTRACT_RATIO"] = prev["CNT_PREV_APPLICATIONS"] / prev["CNT_PREV_CONTRACTS"]

# loan decision ratios
prev = compute_accept_reject_ratio(prev, lags = [1, 3, 5])

# day differences
prev["DAYS_DUE_DIFF_1"] = prev["DAYS_LAST_DUE_1ST_VERSION"] - prev["DAYS_FIRST_DUE"]
prev["DAYS_DUE_DIFF_2"] = prev["DAYS_LAST_DUE"] - prev["DAYS_FIRST_DUE"]
prev["DAYS_TERMINATION_DIFF_1"] = prev["DAYS_TERMINATION"] - prev["DAYS_FIRST_DRAWING"]
prev["DAYS_TERMINATION_DIFF_2"] = prev["DAYS_TERMINATION"] - prev["DAYS_FIRST_DUE"]
prev["DAYS_TERMINATION_DIFF_3"] = prev["DAYS_TERMINATION"] - prev["DAYS_LAST_DUE"]

# application dates
prev["DAY_APPR_PROCESS_START"] = "Working day"
prev["DAY_APPR_PROCESS_START"][(prev["WEEKDAY_APPR_PROCESS_START"] == "SATURDAY") |
                               (prev["WEEKDAY_APPR_PROCESS_START"] == "SUNDAY")] = "Weekend"

In [ ]:
# dummy encodnig for factors
prev = pd.get_dummies(prev, drop_first = True)

In [ ]:
# count missings
nas = count_missings(prev)
nas.head()

# 5. DATA EXPORT

In [ ]:
# merge appl and buro
appl = appl.merge(right = buro.reset_index(), how = "left", on = "SK_ID_CURR")

# merge everything to prev
print(prev.shape)
prev = prev.merge(right = appl.reset_index(), how = "left", on = "SK_ID_PREV")
print(prev.shape)
prev = prev.merge(right = agg_inst.reset_index(), how = "left", on = "SK_ID_CURR")
print(prev.shape)
prev = prev.merge(right = agg_poca.reset_index(), how = "left", on = "SK_ID_CURR")
print(prev.shape)
prev = prev.merge(right = agg_card.reset_index(), how = "left", on = "SK_ID_CURR")
print(prev.shape)

In [ ]:
# export CSV
#prev.to_csv("../data/prepared/home_credit.csv", index = False, float_format = "%.8f")